In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:
# Define the VAE architecture
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Encoder: input -> Latent space
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.ReLU(),
            nn.Linear(400, 20) # Outputs both mean and Log(variance)
        )

        # Separate Layers for the mean and Log-variance
        self.fc_mu = nn.Linear(400, 20)
        self.fc_logvar = nn.Linear(400, 20)

        # Decoder: Latent space -> output
        self.decoder = nn.Sequential(
            nn.Linear(20, 400),
            nn.ReLU(),
            nn.Linear(400, 28*28),
            nn.Sigmoid()
        )

        def encode(self, x):
            h = torch.relu(self.encoder(x))
            mu = self.fc_mu(h)
            logvar = self.fc_logvar(h)

        def reparameterize(self, mu, logvar):
            std = torch.exp(0.5*logvar) # Convert Log(variance) to standard deviation
            eps = torch.randn_like(std) # Sample from a standard normal distribution
            return mu + eps*std # Reparameterization trick

        def decode(self, z):
            return self.decoder(z)

        def forward(self, x):
            mu, logvar = self.encode(x)
            z = self.reparameterize(mu, logvar)
            return self.decode(z), mu, logvar